In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gget

In [2]:
geneNamesPath = "/scratch/indikar_root/indikar1/shared_data/directRNA/references/geneNames.csv"

gf = pd.read_csv(geneNamesPath)
print(f"{gf.shape=}")
print(gf['geneId'].nunique())

gf = gf[['geneId', 'geneName']].drop_duplicates()
print(f"{gf.shape=}")

gf.head()

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/indikar_root/indikar1/shared_data/directRNA/references/geneNames.csv'

In [ ]:
dirPath = "/scratch/indikar_root/indikar1/shared_data/directRNA/counts/"

df = gf.copy()

fileNames = []

for f in os.listdir(dirPath):
    if "total" in f:
        fileId = f.split(".")[0]
        fullPath = f"{dirPath}{f}"
        tf = pd.read_csv(fullPath, header=None, sep="\t")
        tf.columns = ['geneId', fileId]
        fileNames.append(fileId)

        df = pd.merge(df, tf, 
                      how='left',
                      left_on='geneId',
                      right_on='geneId')
    
print(f"{df.shape=}")
df = df.fillna(0)
df.head()

In [ ]:
df['GeneCount'] = df[fileNames].sum(axis=1)
df.head()

In [ ]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.facecolor'] = "w"
plt.rcParams['figure.figsize'] = 3, 1.5

df['GeneCount'].hist(edgecolor='k')
plt.yscale("log")
plt.ylabel('Count')
plt.xlabel('N Genes')
plt.grid(False)

In [ ]:
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.facecolor'] = "w"
plt.rcParams['figure.figsize'] = 4, 3

res = []
for t in np.linspace(0, 1000, 100):
    row = {
        'Threshold' : t,
        'N Genes' : df[df['GeneCount'] > t].shape[0]
    }
    res.append(row)
    
    
res = pd.DataFrame(res)


plt.plot(res['Threshold'], 
         res['N Genes'])

plt.yscale("log")
plt.ylabel('N Genes')
plt.xlabel('Read Count Threshold')
# detected = df[df['GeneCount'] > 0]
# detected.shape

In [ ]:
# t = 0
# genes = df[df['GeneCount'] > t]['geneName'].to_list()
# genes

In [ ]:
t = 0
genes = df[df['GeneCount'] > t]['geneName'].to_list()
print(len(genes))

db = 'PanglaoDB_Augmented_2021'
ef = gget.enrichr(genes, database=db)
pd.set_option('display.max_colwidth', 200)

ef[['path_name', 'p_val', 'overlapping_genes']].head(10)

In [ ]:
alpha = 0.05
ef  = ef[ef['p_val'] <= alpha]
ef.to_csv("CellTypePredictions.csv", index=False)
